# Feature engineering and multi-variable linear regression 

In this notebook we will be looking into multi-variable linear regression. The `LinearRegression` object that we created in the last lab remains the same, but by changing the features that we train it on we can improve its accuracy.

The focus for this notebook then, is primarily about feature engineering and input pipelines. As a result we'll be making a wrapper to train and evaluate a model, given an input pipeline.

But first, some boilerplate.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split

def load_housing_data(test_size=0.2, random_state=None):
    # Load data from Eliiza's github page
    raw_data = pd.read_csv("https://raw.githubusercontent.com/eliiza/ml-training-data/master/housing_price_data/housing_data.csv") 

    # Separate labels from feature columns.
    X = raw_data.drop('SalePrice', axis=1)
    y = raw_data['SalePrice']
    
    # Split the dataset with the requested proportions.
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)
    
    # Return in standard order.
    return (X_train, y_train), (X_test, y_test)

(X_train, y_train), (X_test, y_test) = load_housing_data(random_state=100)

## Evaluating model performance

Depending on the features we feed in to the model, we will see different results. Throughout this notebook we'll call the following method to train and evaluate each model and input pipeline.

This method will do the following:

1. Load the dataset.
1. "Fit" the input pipeline, and apply the transformation to the training data.
1. Train the model on the training dataset.
1. Feed the test data through the input pipeline.
1. Make predictions on the processed test data.
1. Calculate and return the Mean Absolute Error.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

def train_and_evaluate_model(pipeline, model=LinearRegression()):
    # Load the data
    (X_train, y_train), (X_test, y_test) = load_housing_data(random_state=100)
    
    # Prepare the input pipeline and training data
    X_train_prepared = pipeline.fit_transform(X_train)
    
    # Train the model
    model.fit(X_train_prepared, y_train)
    
    # Prepare the test data
    X_test_processed = pipeline.transform(X_test)
    
    # Make some predictions
    y_pred = model.predict(X_test_processed)
    
    # Calculate the error
    mae = mean_absolute_error(y_test, y_pred)
    
    return mae

## Review the dataset

Next we will review the dataset once again. 

In [ ]:
# Show all columns
pd.set_option('display.max_columns', None)

X_train.describe()

### Exercise/Discussion
* Have a look at the count for `LotFrontage` - why is it lower and what should we do about it?
* `MasVnrArea` is missing 6 out of 1168 values. What if that ratio is 1 in 2000 and the missing value was in the test set? Or production data.

Likewise if we look at a categorical column (not displayed above) we'll see that there are some missing values. What should we do with that value when training and making predictions?

In [ ]:
X_train['Electrical'].describe()

# Add a new feature

We'll start off by adding an extra feature without any empty values.

### Exercise

Add another feature to the model - `GrLivArea`. We'll use a `ColumnTransformer` which in its simplest form will filter out our selected columns.

[ColumnTransformer Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html)

In [ ]:
from sklearn.compose import ColumnTransformer

numeric_features = ['OverallQual']

pipeline = ColumnTransformer([
    ('numeric_pipeline', 'passthrough', numeric_features)
])

train_and_evaluate_model(pipeline)

### Exercise

Now try adding a feature with `null`/`NaN` values, such as `LotFrontage`. What happens?

In [ ]:
numeric_features = ['OverallQual', 'GrLivArea']

pipeline = ColumnTransformer([
    ('numeric_pipeline', 'passthrough', numeric_features)
])

train_and_evaluate_model(pipeline)

# Imputing values

If a column is missing values, SciKit Learn provides an object called a `SimpleImputer` that is able to fill in missing values automatically, based on the training data and a user determined `strategy`. Possible strategies for numeric data are `median`, `mean`, or `constant` (with a constant value provided). For categorical types, `most_frequent` is generally a good choice, although `constant` will often work as well.

The code below will replace any missing values with the median value of the training set.

[SimpleImputer Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html)

**Note:** there is no harm in applying this to the other columns.<br>
**Note2:** [Research](https://www.hilarispublisher.com/open-access/a-comparison-of-six-methods-for-missing-data-imputation-2155-6180-1000224.pdf) has shown, that supervised learning methods allow for superior imputations.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

numeric_features = ['OverallQual', 'GrLivArea', 'LotFrontage']

# Create a pipeline for numeric features
numeric_pipeline = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='median')),
])

# Pass only numeric features to the numeric pipeline. Later we will feed different features to different pipelines.
pipeline = ColumnTransformer([
    ('numeric_pipeline', numeric_pipeline, numeric_features)
])

train_and_evaluate_model(pipeline)

### Exercise/Discussion

There are three possible median values we could use to fill in the missing values: 

- The median of the training set
- The median of the test
- The median of the entire dataset.

Which one should we use? Lets have a look, first get the values of each median.

In [ ]:
# Get NaN values from the test set.
nan_entries = X_test[np.isnan(X_test.LotFrontage)]

# Combine train and test sets
X_all = pd.concat([X_train, X_test])

# Fit the pipeline:
pipeline.fit(X_train)

# Print some stats:
print("Train median:", X_train.LotFrontage.median())
print("Test median: ", X_test.LotFrontage.median())
print("Comb. median:", X_all.LotFrontage.median())

Before running the next block, guess what value will be used.

In [ ]:
print("Input:")
print(nan_entries[0:5][numeric_features])

# Apply the transformation to the null entries:
print("\n\nOutput:")
pipeline.transform(nan_entries[0:5])

# Scaling inputs

It is common for ML algorithms to have a preference for scaled inputs - if numbers are very big with a big variance it is possible they will dominate the model in some way.

### Exercise
A benefit of using pipelines is that it is easy to chain together input transformations. Scikit Learn provides a StandardScaler class that will examine a dataset, and figure out how to transform it to make the mean 0, and standard deviation 1. try adding a `StandardScaler` to the pipeline below. 

- Does it make a difference to the model performance?
- Does it matter if it goes before/after the imputer? Why/why not?

[StandardScaler Documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html)

In [ ]:
from sklearn.preprocessing import StandardScaler

numeric_features = ['OverallQual', 'GrLivArea', 'LotFrontage']

numeric_pipeline = Pipeline([
    ('simple_imputer', SimpleImputer(strategy='median'))
])

pipeline = ColumnTransformer([
    ('numeric_pipeline', numeric_pipeline, numeric_features)
])

train_and_evaluate_model(pipeline)

# What does the output look like?

Let's take a look at the output data after the transformations above have been applied. The goal is to achieve the following properties:

* No missing values - counts should all be equal.
* Mean of ~0
* Std Deviation of ~1

The input data denerally won't.

In [ ]:
X_test[numeric_features].describe()

So lets apply our input pipeline and look at the results.

In [ ]:
numeric_pipeline.fit(X_train[numeric_features])

processed = numeric_pipeline.transform(X_test[numeric_features])

pd.DataFrame(processed, columns=numeric_features).describe()

# Categorical Variables

So far we have only looked at numerical values. But what do we do if we have a categorical feature. One example is the `CentralAir` column which has values of `Y` or `N`, indicating whether a property has Central Air Conditioning.

To be useful in a linear regression model we need to convert this into a numeric value. This can be achieved in a number of ways, e.g. `'N' = 0` and `'Y' = 1`, you could also use `-1` and `1`. 

The most important detail is to apply the same transformation on the training data as the test and production data. Once again, `fit`-ing a pipeline can help ensure we achieve this.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

aircon_feature = ['CentralAir']

aircon_encoder = OrdinalEncoder()
aircon_encoder.fit(X_train[aircon_feature])

print(aircon_encoder.categories_)

aircon_encoder.transform(X_test[aircon_feature])

In practice we would add the `OrdinalEncoder` to our pipeline, like so:

In [ ]:
ordinal_features = ['CentralAir']

pipeline = ColumnTransformer([
    ('numerical', numeric_pipeline, numeric_features),
    ('ordinal_encoder', OrdinalEncoder(), ordinal_features)
])

# Feed in unfiltered test data.
train_and_evaluate_model(pipeline)

# Ordinal variables

The example above used the `OrdinalEncoder` to convert from a category to a `0` or a `1`, which isn't a great example of an ordinal variable. A better example is the `BsmtCond` variable, which has a range of values based on the condition:

    BsmtCond: Evaluates the general condition of the basement

       Ex	Excellent
       Gd	Good
       TA	Typical - slight dampness allowed
       Fa	Fair - dampness or some cracking or settling
       Po	Poor - Severe cracking, settling, or wetness

As discussed in the slides there are two options to encode this, One Hot and Ordinal. In this specific case we want to ensure our model knows that these values have an order - that is: Excellent > Good > Typical etc. One hot encoding won't capture this information so an ordinal encoding is prefered.

We also have to deal with situations where there is no basement, and `BsmtCond is NaN`. For this we will use a `SimpleImputer` as above.

Another challenge is that we can't trust the `OrdinalEncoder` to figure out the correct order of the values, we need to specify this directly.

In [ ]:
X_train['BsmtCond'].astype('category').cat.categories

**NOTE:** Not all categories are present. We will still include `Ex` since it is a valid value.

In [ ]:
ordinal_features = [
    'CentralAir',
    'BsmtCond'
]

ordinal_categories = [
    ['N', 'Y'],                     # CentralAir
    ['Po', 'Fa', 'TA', 'Gd', 'Ex']  # BsmtCond
]

ordinal_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('ordinal_encoder', OrdinalEncoder(categories=ordinal_categories))
])

Combine the pipelines using a `ColumnTransformer`.

In [ ]:
pipeline = ColumnTransformer([
    ('numeric_pipeline', numeric_pipeline, numeric_features),
    ('ordinal_pipeline', ordinal_pipeline, ordinal_features)
])

train_and_evaluate_model(pipeline)

## Exercise

Add KitchenQuality as a feature to the code above: see `KitchenQual` in [data_description.txt](https://github.com/eliiza/ml-training-data/blob/master/housing_price_data/data_description.txt)

# One-hot encoding

If there is no order, we use a technique called 'one hot encoding'. This involves creating a new column for each category. This is useful because we specifically want to avoid our model assuming that there is some ordering to the data.

For example, the `Electrical` variable in the housing prices dataset contains the following categories:

       SBrkr	Standard Circuit Breakers & Romex
       FuseA	Fuse Box over 60 AMP and all Romex wiring (Average)	
       FuseF	60 AMP Fuse Box and mostly Romex wiring (Fair)
       FuseP	60 AMP Fuse Box and mostly knob & tube wiring (poor)
       Mix	Mixed

Each row of our data would then have a 1 in the column corresponding to the value in the `Electrical`, and a 0 in all other columns.

|FuseA |FuseF|FuseP|Mix|SBrkr| 
|-:-|-:-|-:-|-:-|-:-|
| 1| 0 | 0 | 0 | 0 |
| 0|1  | 0 | 0 | 0 |
| 0|0  | 1 | 0 | 0 |
| 0|0  | 0 | 1 | 0 |
| 0|0  | 0 | 0 | 1 |

In [ ]:
from sklearn.preprocessing import OneHotEncoder

one_hot_features = ['Electrical']

one_hot_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy="most_frequent")),
    ('one_hot_encoder', OneHotEncoder())
])

In [ ]:
pipeline = ColumnTransformer([
    ('numeric_pipeline', numeric_pipeline, numeric_features),
    ('ordinal_pipeline', ordinal_pipeline, ordinal_features),
    ('one_hot_pipeline', one_hot_pipeline, one_hot_features)
])

train_and_evaluate_model(pipeline)

## Exericse

Try adding `Heating` and/or `Neighborhood` to the one hot encoding pipeline.

# Combining features

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

combine_features = ['FullBath', 'BedroomAbvGr']

pipeline = ColumnTransformer([
    ('numeric_pipeline', numeric_pipeline, numeric_features),
    ('ordinal_pipeline', ordinal_pipeline, ordinal_features),
    ('one_hot_pipeline', one_hot_pipeline, one_hot_features),
    ('combine_features', PolynomialFeatures(interaction_only=True), combine_features)
])

train_and_evaluate_model(pipeline)

# Competition Exercise (optional)

Build a linear model to achieve as low score as possible. See if you can get under $20,000 MAE